Обработка test_1_grup_Ivan

In [1]:
import os

# Пути к папкам
png_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\images"  # Папка с PNG
json_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\json"  # Папка с JSON

# Получаем базовые имена файлов (без расширений)
png_files = {os.path.splitext(f)[0] for f in os.listdir(png_dir) if f.endswith(".png")}
json_files = {os.path.splitext(f)[0] for f in os.listdir(json_dir) if f.endswith(".json")}

# Выявляем лишние файлы
extra_png_files = png_files - json_files  # PNG, которых нет в JSON
extra_json_files = json_files - png_files  # JSON, которых нет в PNG

# Вывод результата
print(f"Лишние PNG файлы ({len(extra_png_files)}):", extra_png_files)
print(f"Лишние JSON файлы ({len(extra_json_files)}):", extra_json_files)

# Записать результат в файлы для анализа
with open("extra_png_files.txt", "w") as f:
    f.write("\n".join(sorted(extra_png_files)))

with open("extra_json_files.txt", "w") as f:
    f.write("\n".join(sorted(extra_json_files)))

print("Проверка завершена. Результаты записаны в extra_png_files.txt и extra_json_files.txt.")

Лишние PNG файлы (0): set()
Лишние JSON файлы (0): set()
Проверка завершена. Результаты записаны в extra_png_files.txt и extra_json_files.txt.


In [2]:
# Предобработка данных
import os
import json

# Папки
json_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\json"  # Путь к папке с JSON
images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\images"    # Путь к изображениям
output_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\labels"  # Папка для сохранения YOLO разметки

# Словарь классов
class_mapping = {
    "title": 0,
    "paragraph": 1,
    "table": 2,
    "picture": 3,
    "table_signature": 4,
    "picture_signature": 5,
    "numbered_list": 6,
    "marked_list": 7,
    "header": 8,
    "footer": 9,
    "footnote": 10,
    "formula": 11
}

# Создаем выходную папку
os.makedirs(output_dir, exist_ok=True)

# Функция для нормализации координат
def normalize_bbox(bbox, img_width, img_height):
    x1, y1, x2, y2 = bbox
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

# Проход по JSON файлам
for json_file in os.listdir(json_dir):
    if json_file.endswith(".json"):
        # Открываем JSON файл
        with open(os.path.join(json_dir, json_file), "r") as f:
            data = json.load(f)
        
        img_width = data["image_width"]
        img_height = data["image_height"]
        txt_filename = os.path.splitext(json_file)[0] + ".txt"
        txt_path = os.path.join(output_dir, txt_filename)
        
        # Запись данных в YOLO формат
        with open(txt_path, "w") as txt_file:
            for class_name, bboxes in data.items():
                if class_name in class_mapping:
                    class_id = class_mapping[class_name]
                    for bbox in bboxes:
                        x_center, y_center, width, height = normalize_bbox(bbox, img_width, img_height)
                        txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [4]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import average_precision_score
import numpy as np

# Пути
input_images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\images"  # Папка с изображениями документов
labels_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\labels"  # Папка с разметкой YOLO
output_images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\\test_results"  # Папка для аннотированных изображений

# Создаём папку для аннотированных изображений, если она не существует
os.makedirs(output_images_dir, exist_ok=True)

# Загружаем модель YOLO
model = YOLO("C:\\Users\\Vito\\Jupyter\\Models\\runs YOLO11s datasetv3 aug imgsz=960\\detect\\train\\weights\\best.pt")

# Настройка кастомных порогов для каждого класса
custom_confidences = {
    0: 0.3,  # title
    1: 0.1,  # paragraph
    9: 0.1, # footer
    11: 0.1, # formula
    10: 0.8, # footnote
    6: 0.4, # numbered_list
    7: 0.4  # marked_list
}

# Список всех изображений в папке
image_files = [f for f in os.listdir(input_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()  # Сортируем для упорядоченности

# 1. Обрабатываем изображения моделью YOLO и визуализируем предсказания с кастомными порогами
annotated_images = []
y_true, y_pred = [], []  # Для метрик

for image_file in image_files:
    image_path = os.path.join(input_images_dir, image_file)
    label_path = os.path.join(labels_dir, Path(image_file).stem + ".txt")  # Соответствующий файл разметки
    
    img = cv2.imread(image_path)

    # Применяем модель YOLO
    results = model.predict(source=img, conf=0.4, iou=0.51, save=False)
    
    # Фильтруем предсказания на основе кастомных порогов для классов
    filtered_boxes = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls)  # Класс объекта
            conf = float(box.conf)  # Уверенность объекта

            # Проверяем, превышает ли уверенность кастомный порог для данного класса
            if cls in custom_confidences:
                if conf >= custom_confidences[cls]:
                    filtered_boxes.append(box)
            else:
                # Используем общий порог (например, 0.4) для классов, для которых не задан кастомный порог
                if conf >= 0.4:
                    filtered_boxes.append(box)

        # Применяем отфильтрованные боксы для визуализации
        result.boxes = filtered_boxes

    # Наносим рамки на изображение
    annotated_img = results[0].plot()  # Визуализируем предсказания (аннотированное изображение)
    annotated_image_path = os.path.join(output_images_dir, image_file)
    cv2.imwrite(annotated_image_path, annotated_img)
    annotated_images.append(annotated_image_path)

    # Сравниваем с разметкой для метрик
    if os.path.exists(label_path):
        # Чтение разметки (ground truth)
        with open(label_path, "r") as f:
            true_boxes = [line.strip().split() for line in f.readlines()]
            true_classes = [int(box[0]) for box in true_boxes]  # Классы из разметки

        # Собираем предсказания
        pred_classes = [int(box.cls) for box in filtered_boxes]

        # Добавляем в список для метрик
        y_true.extend(true_classes)
        y_pred.extend(pred_classes)

# 2. Рассчитываем метрики
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
mIoU = average_precision_score(y_true, y_pred)  # Оценка mAP50-95 (приближённая)
fitness = (precision + recall + f1) / 3  # Условная метрика для общего качества

# Вывод метрик
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print(f"mIoU (mAP50-95): {mIoU:.6f}")
print(f"Fitness: {fitness:.6f}")


0: 768x960 1 paragraph, 261.9ms
Speed: 9.0ms preprocess, 261.9ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 paragraph, 1 table, 264.3ms
Speed: 8.0ms preprocess, 264.3ms inference, 2.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 paragraph, 274.7ms
Speed: 8.0ms preprocess, 274.7ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 960)

0: 960x704 1 title, 1 numbered_list, 256.0ms
Speed: 9.1ms preprocess, 256.0ms inference, 1.3ms postprocess per image at shape (1, 3, 960, 704)

0: 960x768 5 paragraphs, 1 table, 2 numbered_lists, 1 header, 286.8ms
Speed: 8.9ms preprocess, 286.8ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 4 paragraphs, 1 table, 3 marked_lists, 1 header, 265.3ms
Speed: 11.9ms preprocess, 265.3ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 3 paragraphs, 2 tables, 1 numbered_list, 1 marked_list, 1 header, 270.2ms
Speed: 8.3ms preprocess, 270.2m

ValueError: Found input variables with inconsistent numbers of samples: [6847, 5343]

In [5]:
import os

# Путь к тестовому набору данных
test_data_path = r"C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan"

# Проверяем существование необходимых папок
if not os.path.exists(os.path.join(test_data_path, "images")):
    raise FileNotFoundError(f"Папка с изображениями {os.path.join(test_data_path, 'images')} не найдена.")
if not os.path.exists(os.path.join(test_data_path, "labels")):
    raise FileNotFoundError(f"Папка с разметкой {os.path.join(test_data_path, 'labels')} не найдена.")

# Формируем YAML-строку с описанием тестового набора
test_data_yaml = (
    "path: " + test_data_path.replace("\\", "/") + "\n"
    "train: \"\"  # Пустой путь, так как обучение не требуется\n"
    "val: images\n"
    "names:\n"
    "  0: title\n"
    "  1: paragraph\n"
    "  2: table\n"
    "  3: picture\n"
    "  4: table_signature\n"
    "  5: picture_signature\n"
    "  6: numbered_list\n"
    "  7: marked_list\n"
    "  8: header\n"
    "  9: footer\n"
    "  10: footnote\n"
    "  11: formula\n"
)

# Сохраняем YAML-файл
yaml_path = os.path.join(test_data_path, "test.yaml")
with open(yaml_path, "w", encoding="utf-8") as file:
    file.write(test_data_yaml)

print(f"Файл YAML для тестового набора сохранён в: {yaml_path}")

# Выполняем валидацию на тестовом наборе
results = model.val(data=yaml_path)

# Получаем метрики из results_dict
metrics = results.results_dict

# Извлекаем precision и recall
precision = metrics['metrics/precision(B)']
recall = metrics['metrics/recall(B)']

# Вычисляем F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

# Извлекаем mIoU (mAP50-95)
mAP50_95 = metrics['metrics/mAP50-95(B)']

# Печатаем метрики
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("mIoU (mAP50-95):", mAP50_95)
print("Fitness:", metrics['fitness'])

Файл YAML для тестового набора сохранён в: C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Ivan\test.yaml
Ultralytics 8.3.34  Python-3.11.9 torch-2.5.1+cpu CPU (AMD Ryzen 7 7840HS w/ Radeon 780M Graphics)


val: Scanning C:\Users\Vito\Jupyter\Models\test_1_grup_Ivan\labels... 1000 images, 0 backgrounds, 0 corrupt: 100%|█████


val: New cache created: C:\Users\Vito\Jupyter\Models\test_1_grup_Ivan\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [06:19


                   all       1000       6847      0.498      0.405      0.502      0.403
                 title        444        571      0.049     0.0368    0.00893    0.00712
             paragraph        854       2289       0.51      0.616      0.605      0.373
                 table        338        379      0.758      0.871      0.895      0.658
               picture        240        241      0.292      0.622      0.407      0.151
       table_signature        251        272       0.17    0.00368      0.154       0.14
     picture_signature        240        241      0.156    0.00415      0.104      0.088
         numbered_list        299        341       0.88      0.754      0.825      0.774
           marked_list        241        284       0.68      0.972      0.955      0.883
                header        574        574      0.997      0.606      0.988      0.908
                footer        691        691          1      0.361      0.869      0.677
              footnot

Обработка test_1_grup_Tim

In [7]:
import os

# Пути к папкам
png_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\image"  # Папка с PNG
json_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\json"  # Папка с JSON

# Получаем базовые имена файлов (без расширений)
png_files = {os.path.splitext(f)[0] for f in os.listdir(png_dir) if f.endswith(".png")}
json_files = {os.path.splitext(f)[0] for f in os.listdir(json_dir) if f.endswith(".json")}

# Выявляем лишние файлы
extra_png_files = png_files - json_files  # PNG, которых нет в JSON
extra_json_files = json_files - png_files  # JSON, которых нет в PNG

# Вывод результата
print(f"Лишние PNG файлы ({len(extra_png_files)}):", extra_png_files)
print(f"Лишние JSON файлы ({len(extra_json_files)}):", extra_json_files)

# Записать результат в файлы для анализа
with open("extra_png_files.txt", "w") as f:
    f.write("\n".join(sorted(extra_png_files)))

with open("extra_json_files.txt", "w") as f:
    f.write("\n".join(sorted(extra_json_files)))

print("Проверка завершена. Результаты записаны в extra_png_files.txt и extra_json_files.txt.")

Лишние PNG файлы (16): {'document_70_page_3_annotated', 'document_70_page_4_annotated', 'document_71_page_6_annotated', 'document_71_page_3_annotated', 'document_70_page_1_annotated', 'document_70_page_8_annotated', 'document_71_page_5_annotated', 'document_71_page_4_annotated', 'document_70_page_7_annotated', 'document_70_page_5_annotated', 'document_71_page_2_annotated', 'document_70_page_2_annotated', 'document_70_page_9_annotated', 'document_70_page_10_annotated', 'document_71_page_1_annotated', 'document_70_page_6_annotated'}
Лишние JSON файлы (16): {'document_71_page_1', 'document_71_page_4', 'document_71_page_2', 'document_70_page_5', 'document_70_page_8', 'document_71_page_3', 'document_70_page_4', 'document_71_page_5', 'document_70_page_2', 'document_70_page_10', 'document_71_page_6', 'document_70_page_3', 'document_70_page_9', 'document_70_page_7', 'document_70_page_6', 'document_70_page_1'}
Проверка завершена. Результаты записаны в extra_png_files.txt и extra_json_files.txt.

In [8]:
import os

# Пути к папкам
png_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\image"  # Папка с PNG
json_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\json"  # Папка с JSON

# Файлы с лишними элементами
extra_png_file = "extra_png_files.txt"
extra_json_file = "extra_json_files.txt"

# Функция удаления файлов
def delete_files(file_list, folder_path):
    with open(file_list, "r") as f:
        files_to_delete = f.read().splitlines()
    
    for file_name in files_to_delete:
        file_path = os.path.join(folder_path, f"{file_name}.png") if "png" in file_list else os.path.join(folder_path, f"{file_name}.json")
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Удалено: {file_path}")
        else:
            print(f"Файл не найден (пропущено): {file_path}")

# Удаляем лишние PNG файлы
delete_files(extra_png_file, png_dir)

# Удаляем лишние JSON файлы
delete_files(extra_json_file, json_dir)

print("Удаление завершено.")

Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_10_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_1_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_2_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_3_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_4_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_5_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_6_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_7_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_8_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_70_page_9_annotated.png
Удалено: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\image\document_

In [9]:
# Предобработка данных
import os
import json

# Папки
json_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\json"  # Путь к папке с JSON
images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\image"    # Путь к изображениям
output_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\labels"  # Папка для сохранения YOLO разметки

# Словарь классов
class_mapping = {
    "title": 0,
    "paragraph": 1,
    "table": 2,
    "picture": 3,
    "graph": 3,
    "table_signature": 4,
    "picture_signature": 5,
    "numbered_list": 6,
    "marked_list": 7,
    "header": 8,
    "footer": 9,
    "footnote": 10,
    "formula": 11
}

# Создаем выходную папку
os.makedirs(output_dir, exist_ok=True)

# Функция для нормализации координат
def normalize_bbox(bbox, img_width, img_height):
    x1, y1, x2, y2 = bbox
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

# Проход по JSON файлам
for json_file in os.listdir(json_dir):
    if json_file.endswith(".json"):
        # Открываем JSON файл
        with open(os.path.join(json_dir, json_file), "r") as f:
            data = json.load(f)
        
        img_width = data["image_width"]
        img_height = data["image_height"]
        txt_filename = os.path.splitext(json_file)[0] + ".txt"
        txt_path = os.path.join(output_dir, txt_filename)
        
        # Запись данных в YOLO формат
        with open(txt_path, "w") as txt_file:
            for class_name, bboxes in data.items():
                if class_name in class_mapping:
                    class_id = class_mapping[class_name]
                    for bbox in bboxes:
                        x_center, y_center, width, height = normalize_bbox(bbox, img_width, img_height)
                        txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [12]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import average_precision_score
import numpy as np

# Пути
input_images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\image"  # Папка с изображениями документов
labels_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\labels"  # Папка с разметкой YOLO
output_images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\\test_results"  # Папка для аннотированных изображений

# Создаём папку для аннотированных изображений, если она не существует
os.makedirs(output_images_dir, exist_ok=True)

# Загружаем модель YOLO
model = YOLO("C:\\Users\\Vito\\Jupyter\\Models\\runs YOLO11s datasetv3 aug imgsz=960\\detect\\train\\weights\\best.pt")

# Настройка кастомных порогов для каждого класса
custom_confidences = {
    0: 0.3,  # title
    1: 0.1,  # paragraph
    9: 0.15, # footer
    11: 0.1, # formula
    10: 0.88, # footnote
    6: 0.4, # numbered_list
    7: 0.4  # marked_list
}

# Список всех изображений в папке
image_files = [f for f in os.listdir(input_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()  # Сортируем для упорядоченности

# 1. Обрабатываем изображения моделью YOLO и визуализируем предсказания с кастомными порогами
annotated_images = []
y_true, y_pred = [], []  # Для метрик

for image_file in image_files:
    image_path = os.path.join(input_images_dir, image_file)
    label_path = os.path.join(labels_dir, Path(image_file).stem + ".txt")  # Соответствующий файл разметки
    
    img = cv2.imread(image_path)

    # Применяем модель YOLO
    results = model.predict(source=img, conf=0.4, iou=0.51, save=False)
    
    # Фильтруем предсказания на основе кастомных порогов для классов
    filtered_boxes = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls)  # Класс объекта
            conf = float(box.conf)  # Уверенность объекта

            # Проверяем, превышает ли уверенность кастомный порог для данного класса
            if cls in custom_confidences:
                if conf >= custom_confidences[cls]:
                    filtered_boxes.append(box)
            else:
                # Используем общий порог (например, 0.4) для классов, для которых не задан кастомный порог
                if conf >= 0.4:
                    filtered_boxes.append(box)

        # Применяем отфильтрованные боксы для визуализации
        result.boxes = filtered_boxes

    # Наносим рамки на изображение
    annotated_img = results[0].plot()  # Визуализируем предсказания (аннотированное изображение)
    annotated_image_path = os.path.join(output_images_dir, image_file)
    cv2.imwrite(annotated_image_path, annotated_img)
    annotated_images.append(annotated_image_path)

    # Сравниваем с разметкой для метрик
    if os.path.exists(label_path):
        # Чтение разметки (ground truth)
        with open(label_path, "r") as f:
            true_boxes = [line.strip().split() for line in f.readlines()]
            true_classes = [int(box[0]) for box in true_boxes]  # Классы из разметки

        # Собираем предсказания
        pred_classes = [int(box.cls) for box in filtered_boxes]

        # Добавляем в список для метрик
        y_true.extend(true_classes)
        y_pred.extend(pred_classes)

# 2. Рассчитываем метрики
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
mIoU = average_precision_score(y_true, y_pred)  # Оценка mAP50-95 (приближённая)
fitness = (precision + recall + f1) / 3  # Условная метрика для общего качества

# Вывод метрик
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print(f"mIoU (mAP50-95): {mIoU:.6f}")
print(f"Fitness: {fitness:.6f}")


0: 768x960 1 title, 1 paragraph, 1 numbered_list, 1 header, 1 footer, 486.7ms
Speed: 13.0ms preprocess, 486.7ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 paragraph, 1 header, 1 footer, 430.8ms
Speed: 20.5ms preprocess, 430.8ms inference, 3.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 2 paragraphs, 1 table, 1 header, 1 footer, 393.5ms
Speed: 16.0ms preprocess, 393.5ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 4 paragraphs, 1 numbered_list, 1 header, 1 footer, 2 footnotes, 298.8ms
Speed: 13.0ms preprocess, 298.8ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 paragraph, 1 header, 1 footer, 236.3ms
Speed: 7.1ms preprocess, 236.3ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 2 titles, 4 paragraphs, 2 pictures, 1 header, 1 footer, 327.0ms
Speed: 14.6ms preprocess, 327.0ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 960)

ValueError: Found input variables with inconsistent numbers of samples: [5789, 4425]

In [14]:
import os

# Путь к тестовому набору данных
test_data_path = r"C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim"

# Проверяем существование необходимых папок
if not os.path.exists(os.path.join(test_data_path, "images")):
    raise FileNotFoundError(f"Папка с изображениями {os.path.join(test_data_path, 'images')} не найдена.")
if not os.path.exists(os.path.join(test_data_path, "labels")):
    raise FileNotFoundError(f"Папка с разметкой {os.path.join(test_data_path, 'labels')} не найдена.")

# Формируем YAML-строку с описанием тестового набора
test_data_yaml = (
    "path: " + test_data_path.replace("\\", "/") + "\n"
    "train: \"\"  # Пустой путь, так как обучение не требуется\n"
    "val: images\n"
    "names:\n"
    "  0: title\n"
    "  1: paragraph\n"
    "  2: table\n"
    "  3: picture\n"
    "  4: table_signature\n"
    "  5: picture_signature\n"
    "  6: numbered_list\n"
    "  7: marked_list\n"
    "  8: header\n"
    "  9: footer\n"
    "  10: footnote\n"
    "  11: formula\n"
)

# Сохраняем YAML-файл
yaml_path = os.path.join(test_data_path, "test.yaml")
with open(yaml_path, "w", encoding="utf-8") as file:
    file.write(test_data_yaml)

print(f"Файл YAML для тестового набора сохранён в: {yaml_path}")

# Выполняем валидацию на тестовом наборе
results = model.val(data=yaml_path)

# Получаем метрики из results_dict
metrics = results.results_dict

# Извлекаем precision и recall
precision = metrics['metrics/precision(B)']
recall = metrics['metrics/recall(B)']

# Вычисляем F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

# Извлекаем mIoU (mAP50-95)
mAP50_95 = metrics['metrics/mAP50-95(B)']

# Печатаем метрики
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("mIoU (mAP50-95):", mAP50_95)
print("Fitness:", metrics['fitness'])

Файл YAML для тестового набора сохранён в: C:\\Users\\Vito\\Jupyter\\Models\\test_1_grup_Tim\test.yaml
Ultralytics 8.3.34  Python-3.11.9 torch-2.5.1+cpu CPU (AMD Ryzen 7 7840HS w/ Radeon 780M Graphics)


val: Scanning C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\labels... 984 images, 0 backgrounds, 0 corrupt: 100%|███████


val: New cache created: C:\Users\Vito\Jupyter\Models\test_1_grup_Tim\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [06:22


                   all        984       5789      0.412      0.393      0.417      0.307
                 title        413        413      0.365      0.446      0.295      0.177
             paragraph        805       1173      0.394      0.747      0.549      0.441
                 table        180        180      0.696      0.611      0.671      0.441
               picture        159        175        0.3      0.971      0.752      0.634
       table_signature        177        177     0.0464    0.00262     0.0619     0.0471
     picture_signature        159        175      0.346     0.0333      0.207      0.144
         numbered_list        429        514       0.37      0.319      0.314      0.261
           marked_list        408        481      0.443      0.233      0.268      0.214
                header        941        941      0.989      0.775      0.978      0.672
                footer        891        891          1      0.584      0.906      0.652
              footnot

In [ ]:
Обработка test_2_grup_Tim

In [15]:
import os

# Пути к папкам
png_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\images"  # Папка с PNG
json_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\json"  # Папка с JSON

# Получаем базовые имена файлов (без расширений)
png_files = {os.path.splitext(f)[0] for f in os.listdir(png_dir) if f.endswith(".png")}
json_files = {os.path.splitext(f)[0] for f in os.listdir(json_dir) if f.endswith(".json")}

# Выявляем лишние файлы
extra_png_files = png_files - json_files  # PNG, которых нет в JSON
extra_json_files = json_files - png_files  # JSON, которых нет в PNG

# Вывод результата
print(f"Лишние PNG файлы ({len(extra_png_files)}):", extra_png_files)
print(f"Лишние JSON файлы ({len(extra_json_files)}):", extra_json_files)

# Записать результат в файлы для анализа
with open("extra_png_files.txt", "w") as f:
    f.write("\n".join(sorted(extra_png_files)))

with open("extra_json_files.txt", "w") as f:
    f.write("\n".join(sorted(extra_json_files)))

print("Проверка завершена. Результаты записаны в extra_png_files.txt и extra_json_files.txt.")

Лишние PNG файлы (0): set()
Лишние JSON файлы (0): set()
Проверка завершена. Результаты записаны в extra_png_files.txt и extra_json_files.txt.


In [16]:
# Предобработка данных
import os
import json

# Папки
json_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\json"  # Путь к папке с JSON
images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\images"    # Путь к изображениям
output_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\labels"  # Папка для сохранения YOLO разметки

# Словарь классов
class_mapping = {
    "title": 0,
    "paragraph": 1,
    "table": 2,
    "picture": 3,
    "graph": 3,
    "table_signature": 4,
    "picture_signature": 5,
    "numbered_list": 6,
    "marked_list": 7,
    "header": 8,
    "footer": 9,
    "footnote": 10,
    "formula": 11
}

# Создаем выходную папку
os.makedirs(output_dir, exist_ok=True)

# Функция для нормализации координат
def normalize_bbox(bbox, img_width, img_height):
    x1, y1, x2, y2 = bbox
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

# Проход по JSON файлам
for json_file in os.listdir(json_dir):
    if json_file.endswith(".json"):
        # Открываем JSON файл
        with open(os.path.join(json_dir, json_file), "r") as f:
            data = json.load(f)
        
        img_width = data["image_width"]
        img_height = data["image_height"]
        txt_filename = os.path.splitext(json_file)[0] + ".txt"
        txt_path = os.path.join(output_dir, txt_filename)
        
        # Запись данных в YOLO формат
        with open(txt_path, "w") as txt_file:
            for class_name, bboxes in data.items():
                if class_name in class_mapping:
                    class_id = class_mapping[class_name]
                    for bbox in bboxes:
                        x_center, y_center, width, height = normalize_bbox(bbox, img_width, img_height)
                        txt_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [19]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import average_precision_score
import numpy as np

# Пути
input_images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\images"  # Папка с изображениями документов
labels_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\labels"  # Папка с разметкой YOLO
output_images_dir = "C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\\test_results"  # Папка для аннотированных изображений

# Создаём папку для аннотированных изображений, если она не существует
os.makedirs(output_images_dir, exist_ok=True)

# Загружаем модель YOLO
model = YOLO("C:\\Users\\Vito\\Jupyter\\Models\\runs YOLO11s datasetv3 aug imgsz=960\\detect\\train\\weights\\best.pt")

# Настройка кастомных порогов для каждого класса
custom_confidences = {
    0: 0.3,  # title
    1: 0.1,  # paragraph
    9: 0.1, # footer
    11: 0.1, # formula
    10: 0.4, # footnote
    6: 0.4, # numbered_list
    7: 0.4  # marked_list
}

# Список всех изображений в папке
image_files = [f for f in os.listdir(input_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()  # Сортируем для упорядоченности

# 1. Обрабатываем изображения моделью YOLO и визуализируем предсказания с кастомными порогами
annotated_images = []
y_true, y_pred = [], []  # Для метрик

for image_file in image_files:
    image_path = os.path.join(input_images_dir, image_file)
    label_path = os.path.join(labels_dir, Path(image_file).stem + ".txt")  # Соответствующий файл разметки
    
    img = cv2.imread(image_path)

    # Применяем модель YOLO
    results = model.predict(source=img, conf=0.4, iou=0.51, save=False)
    
    # Фильтруем предсказания на основе кастомных порогов для классов
    filtered_boxes = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls)  # Класс объекта
            conf = float(box.conf)  # Уверенность объекта

            # Проверяем, превышает ли уверенность кастомный порог для данного класса
            if cls in custom_confidences:
                if conf >= custom_confidences[cls]:
                    filtered_boxes.append(box)
            else:
                # Используем общий порог (например, 0.4) для классов, для которых не задан кастомный порог
                if conf >= 0.4:
                    filtered_boxes.append(box)

        # Применяем отфильтрованные боксы для визуализации
        result.boxes = filtered_boxes

    # Наносим рамки на изображение
    annotated_img = results[0].plot()  # Визуализируем предсказания (аннотированное изображение)
    annotated_image_path = os.path.join(output_images_dir, image_file)
    cv2.imwrite(annotated_image_path, annotated_img)
    annotated_images.append(annotated_image_path)

    # Сравниваем с разметкой для метрик
    if os.path.exists(label_path):
        # Чтение разметки (ground truth)
        with open(label_path, "r") as f:
            true_boxes = [line.strip().split() for line in f.readlines()]
            true_classes = [int(box[0]) for box in true_boxes]  # Классы из разметки

        # Собираем предсказания
        pred_classes = [int(box.cls) for box in filtered_boxes]

        # Добавляем в список для метрик
        y_true.extend(true_classes)
        y_pred.extend(pred_classes)

# 2. Рассчитываем метрики
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
mIoU = average_precision_score(y_true, y_pred)  # Оценка mAP50-95 (приближённая)
fitness = (precision + recall + f1) / 3  # Условная метрика для общего качества

# Вывод метрик
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print(f"mIoU (mAP50-95): {mIoU:.6f}")
print(f"Fitness: {fitness:.6f}")


0: 960x768 1 title, 5 paragraphs, 1 table, 1 footnote, 282.9ms
Speed: 8.9ms preprocess, 282.9ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 2 titles, 3 paragraphs, 1 picture, 1 numbered_list, 291.2ms
Speed: 9.2ms preprocess, 291.2ms inference, 6.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 4 paragraphs, 1 table, 1 numbered_list, 286.8ms
Speed: 5.3ms preprocess, 286.8ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 1 title, 4 paragraphs, 1 picture, 1 numbered_list, 2 marked_lists, 272.0ms
Speed: 7.7ms preprocess, 272.0ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 3 paragraphs, 236.2ms
Speed: 12.5ms preprocess, 236.2ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 2 paragraphs, 275.4ms
Speed: 21.8ms preprocess, 275.4ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 768x960 1 title, 5 paragraphs, 1 picture, 1 header, 1 f

ValueError: Found input variables with inconsistent numbers of samples: [6683, 5586]

In [20]:
import os

# Путь к тестовому набору данных
test_data_path = r"C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim"

# Проверяем существование необходимых папок
if not os.path.exists(os.path.join(test_data_path, "images")):
    raise FileNotFoundError(f"Папка с изображениями {os.path.join(test_data_path, 'images')} не найдена.")
if not os.path.exists(os.path.join(test_data_path, "labels")):
    raise FileNotFoundError(f"Папка с разметкой {os.path.join(test_data_path, 'labels')} не найдена.")

# Формируем YAML-строку с описанием тестового набора
test_data_yaml = (
    "path: " + test_data_path.replace("\\", "/") + "\n"
    "train: \"\"  # Пустой путь, так как обучение не требуется\n"
    "val: images\n"
    "names:\n"
    "  0: title\n"
    "  1: paragraph\n"
    "  2: table\n"
    "  3: picture\n"
    "  4: table_signature\n"
    "  5: picture_signature\n"
    "  6: numbered_list\n"
    "  7: marked_list\n"
    "  8: header\n"
    "  9: footer\n"
    "  10: footnote\n"
    "  11: formula\n"
)

# Сохраняем YAML-файл
yaml_path = os.path.join(test_data_path, "test.yaml")
with open(yaml_path, "w", encoding="utf-8") as file:
    file.write(test_data_yaml)

print(f"Файл YAML для тестового набора сохранён в: {yaml_path}")

# Выполняем валидацию на тестовом наборе
results = model.val(data=yaml_path)

# Получаем метрики из results_dict
metrics = results.results_dict

# Извлекаем precision и recall
precision = metrics['metrics/precision(B)']
recall = metrics['metrics/recall(B)']

# Вычисляем F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

# Извлекаем mIoU (mAP50-95)
mAP50_95 = metrics['metrics/mAP50-95(B)']

# Печатаем метрики
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("mIoU (mAP50-95):", mAP50_95)
print("Fitness:", metrics['fitness'])

Файл YAML для тестового набора сохранён в: C:\\Users\\Vito\\Jupyter\\Models\\test_2_grup_Tim\test.yaml
Ultralytics 8.3.34  Python-3.11.9 torch-2.5.1+cpu CPU (AMD Ryzen 7 7840HS w/ Radeon 780M Graphics)


val: Scanning C:\Users\Vito\Jupyter\Models\test_2_grup_Tim\labels... 1072 images, 0 backgrounds, 0 corrupt: 100%|██████


val: New cache created: C:\Users\Vito\Jupyter\Models\test_2_grup_Tim\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [05:42


                   all       1072       6683      0.563       0.42      0.453      0.311
                 title        414        490      0.429      0.588      0.431      0.281
             paragraph        910       1717       0.46      0.722       0.63      0.509
                 table        282        298      0.772      0.792      0.809       0.59
               picture        250        255       0.39      0.737      0.522      0.312
       table_signature        226        240      0.217    0.00833      0.065     0.0539
     picture_signature        250        255      0.727     0.0706      0.255      0.201
         numbered_list        291        301      0.429      0.332      0.305      0.241
           marked_list        381        399      0.568      0.338      0.416      0.332
                header       1072       1072      0.995      0.713      0.931      0.606
                footer        604        604          1      0.671      0.927      0.507
              footnot

In [ ]:
Обработка test_3_grup_Tim...